# Imports

In [2]:
import streamlit as st
import re
import pandas as pd
from whatstk import df_from_txt_whatsapp
from collections import defaultdict

# Functions Definition

In [3]:
def extract_phone_number(message):
    pattern = r'91(\d{10,15})'
    match = re.search(pattern, message)
    if match:
        return match.group(1)
    return None

def convert_df(df):
    # IMPORTANT: Cache the conversion to prevent computation on every rerun
    return df.to_csv(index=False).encode("utf-8")

def extract_name(message):
    pattern = r'@⁨([^⁩]+)⁩'
    match = re.search(pattern, message)
    if match:
        return match.group(1).strip()
    return None

In [16]:
df = df_from_txt_whatsapp('chats.txt')

C:\Users\P\AppData\Local\Temp\ipykernel_12496\904238115.py:1: FutureWarning:

This function is deprecated and will be removed in future versions. Use `df_from_whatsapp` instead.



In [17]:
Category = 'Friend'

In [6]:
birthday_dict = defaultdict(set)

# Chats Processing

In [18]:
df['Date'] = df['date'].dt.date
birthday_rows = df[df['message'].str.contains('Happy Birthday', na=False)]
for index, row in birthday_rows.iterrows():
    name = extract_name(row['message'])
    if name is not None:
        birthday_dict[name].add(row['Date'])


# Data post processing

In [8]:
birthday_df = pd.DataFrame.from_dict(birthday_dict, orient='index')
out_df = pd.DataFrame({'Name':birthday_df.index,'Birth date':birthday_df[0]})
out_df.reset_index(drop=True, inplace=True)
out_df['Category'] = Category
out_df.to_csv('out.csv',index=False)